# Notebook 0 — Raw Data Inspection & Schema Validation

## Data Provenance & Scope

**Dataset:** MIMIC-IV v3.x (PhysioNet)

This project uses MIMIC-IV electronic health record data.  
Raw tables are sourced from a local MIMIC-IV archive and selectively staged to a compute VM for analysis under infrastructure constraints.

This notebook performs **foundational data validation only**, including:

- Schema inspection (columns, data types)
- Missing value assessment
- Structural sanity checks
- Cross-table consistency verification

No filtering, feature engineering, aggregation, or modeling is performed here.

The purpose of this notebook is to establish **trust in the raw data** and define the structural assumptions used throughout the project.

Once completed, this notebook is **locked** and used strictly as a reference.

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

## File Inspection: patients.csv

### 1. Provenance & Integrity
We verify file location, size, and row count before trusting the data.

In [5]:
patients_path = r"D:\mimic-iv-3.1\Unzipped_raw_data\patients.csv"

patients_df = pd.read_csv(
    patients_path,
    nrows=1000
)

patients_df.shape

(1000, 6)

### Observation — patients.csv shape

- Sampled 1,000 rows successfully
- Total columns: 6
- File loads without errors or truncation

Indicates stable file structure suitable for further inspection.

In [6]:
patients_df.columns

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='str')

### Observation — patients.csv columns

- Columns represent patient identity and demographic context
- No admission-level or treatment-level information present
- Table provides context, not behavioral signals

This table will be used only for linkage and stratification.

In [7]:
patients_df.dtypes

subject_id           int64
gender                 str
anchor_age           int64
anchor_year          int64
anchor_year_group      str
dod                    str
dtype: object

### Schema Inference — patients.csv

- `subject_id`: integer identifier; must never be used as a numeric feature
- `gender`: categorical demographic context
- `anchor_age`: de-identified age; suitable for coarse stratification only
- `anchor_year`: de-identification anchor; not a real calendar year
- `anchor_year_group`: categorical temporal grouping
- `dod`: stored as string; date parsing deferred

No schema violations detected.  
No corrections applied at this stage.

In [8]:
patients_df.isna().sum()

subject_id             0
gender                 0
anchor_age             0
anchor_year            0
anchor_year_group      0
dod                  888
dtype: int64

### Observation — patients.csv missingness

- No missing values in identifiers or demographic fields
- `dod` is missing for the majority of patients
- Missingness in `dod` is expected and reflects censoring, not data quality issues

Missingness is informative and will not be imputed at this stage.

### Red Flags & Decisions — patients.csv

- No schema violations detected
- `subject_id` treated strictly as an identifier
- `anchor_year` and `anchor_year_group` are de-identification artifacts
- `dod` retained for context; not used for prediction

No transformations applied in this notebook.

## File Inspection: admissions.csv

### 1. Provenance & Integrity
We inspect admission-level structure, timestamps, and system-level outcomes.

In [9]:
admissions_path = r"D:\mimic-iv-3.1\Unzipped_raw_data\admissions.csv"

admissions_df = pd.read_csv(
    admissions_path,
    nrows=1000
)
admissions_df.shape

(1000, 16)

### Observation — admissions.csv shape

- Sampled 1,000 admissions successfully
- Total columns: 16
- File loads without parsing errors

Admissions table appears structurally stable for further inspection.

In [10]:
admissions_df.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admit_provider_id', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag'],
      dtype='str')

### Observation — admissions.csv columns

- Table contains admission-level identifiers, timestamps, and system decisions
- Multiple time fields enable length-of-stay and flow analysis
- Demographic and administrative fields provide context, not behavioral signals
- Outcome-related fields (`deathtime`, `hospital_expire_flag`) present but not modeling targets

This table defines the unit of system-level analysis.

In [11]:
admissions_df.dtypes

subject_id              int64
hadm_id                 int64
admittime                 str
dischtime                 str
deathtime                 str
admission_type            str
admit_provider_id         str
admission_location        str
discharge_location        str
insurance                 str
language                  str
marital_status            str
race                      str
edregtime                 str
edouttime                 str
hospital_expire_flag    int64
dtype: object

### Schema Inference — admissions.csv

- Identifier fields (`subject_id`, `hadm_id`) correctly read as integers
- All timestamp fields currently parsed as strings
- Categorical and administrative fields parsed as strings
- `hospital_expire_flag` represented as binary integer

Timestamp parsing is deferred to later preprocessing stages.
No schema corrections applied at this stage.

In [12]:
admissions_df.isna().sum()

subject_id                0
hadm_id                   0
admittime                 0
dischtime                 0
deathtime               980
admission_type            0
admit_provider_id         0
admission_location        0
discharge_location      314
insurance                18
language                  1
marital_status           30
race                      0
edregtime               264
edouttime               264
hospital_expire_flag      0
dtype: int64

### Observation — admissions.csv missingness

- Core identifiers and admission timestamps have no missing values
- `deathtime` is missing for most admissions, consistent with in-hospital mortality being rare
- Discharge-related fields show partial missingness (`discharge_location`)
- ED timestamps (`edregtime`, `edouttime`) missing when admission did not originate in ED
- Minor missingness in administrative context fields (`insurance`, `language`, `marital_status`)

Missingness patterns are structurally meaningful and not indicative of data quality issues.

### Red Flags & Decisions — admissions.csv

- No schema violations detected
- Admission and discharge timestamps retained as strings for now
- ED-related fields interpreted conditionally (ED vs non-ED admissions)
- Outcome-related fields retained for context, not prediction

Admissions table confirmed as primary unit for system-level analysis.

## File Inspection: diagnoses_icd.csv

### 1. Provenance & Integrity
We inspect diagnosis coding structure and multiplicity per admission.

In [13]:
diagnoses_path = r"D:\mimic-iv-3.1\Unzipped_raw_data\diagnoses_icd.csv"

diagnoses_df = pd.read_csv(
    diagnoses_path,
    nrows=1000
)

diagnoses_df.shape

(1000, 5)

### Observation — diagnoses_icd.csv shape

- Sampled 1,000 diagnosis records successfully
- Total columns: 5
- Table is long-form with multiple rows per admission

Structure is consistent with diagnosis multiplicity.

In [14]:
diagnoses_df.columns

Index(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version'], dtype='str')

### Observation — diagnoses_icd.csv columns

- Table links multiple diagnoses to a single admission (`hadm_id`)
- Diagnosis order captured via `seq_num`
- Diagnoses encoded using ICD-9 and ICD-10 (`icd_code`, `icd_version`)
- Multiple rows per admission expected and preserved

This table captures diagnostic multiplicity and coding structure.

In [15]:
diagnoses_df.dtypes

subject_id     int64
hadm_id        int64
seq_num        int64
icd_code         str
icd_version    int64
dtype: object

### Schema Inference — diagnoses_icd.csv

- Identifier fields (`subject_id`, `hadm_id`) correctly parsed as integers
- `seq_num` parsed as integer, preserving diagnosis ordering
- `icd_code` parsed as string
- `icd_version` parsed as integer (ICD-9 vs ICD-10 indicator)

No schema violations detected.
No transformations applied at this stage.

In [16]:
diagnoses_df.isna().sum()

subject_id     0
hadm_id        0
seq_num        0
icd_code       0
icd_version    0
dtype: int64

### Observation — diagnoses_icd.csv missingness

- No missing values detected across all columns
- Diagnosis records are complete at the admission level

No missingness handling required at this stage.

### Red Flags & Decisions — diagnoses_icd.csv

- No schema violations or missingness detected
- Diagnosis order (`seq_num`) preserved
- ICD codes retained as symbolic identifiers
- No aggregation or filtering applied at this stage

This table will be used to model diagnostic multiplicity per admission.


## File Inspection: prescriptions.csv

### 1. Provenance & Integrity
We inspect medication order structure and treatment adaptation signals.

In [18]:
prescriptions_path = r"D:\mimic-iv-3.1\Unzipped_raw_data\prescriptions.csv"

prescriptions_df = pd.read_csv(
    prescriptions_path,
    nrows=1000
)

prescriptions_df.shape

(1000, 21)

### Observation — prescriptions.csv shape

- Sampled 1,000 prescription records successfully
- Total columns: 21
- Table is wide and medication-focused

Structure appears suitable for treatment adaptation analysis.

In [19]:
prescriptions_df.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route'],
      dtype='str')

### Observation — prescriptions.csv columns

- Table captures medication orders linked to admissions (`hadm_id`)
- Multiple identifiers reflect internal order tracking, not clinical signals
- Timing fields (`starttime`, `stoptime`) enable treatment adaptation analysis
- Medication identity encoded via names and standardized codes
- Dose and form fields present but potentially noisy

This table represents treatment intent and adaptation, not confirmed administration.

In [20]:
prescriptions_df.dtypes

subject_id             int64
hadm_id                int64
pharmacy_id            int64
poe_id                   str
poe_seq              float64
order_provider_id        str
starttime                str
stoptime                 str
drug_type                str
drug                     str
formulary_drug_cd        str
gsn                  float64
ndc                    int64
prod_strength            str
form_rx              float64
dose_val_rx              str
dose_unit_rx             str
form_val_disp            str
form_unit_disp           str
doses_per_24_hrs     float64
route                    str
dtype: object

### Schema Inference — prescriptions.csv

- Core identifiers (`subject_id`, `hadm_id`, `pharmacy_id`) parsed as integers
- Order tracking fields include mixed types (`poe_seq`, `gsn`, `form_rx`)
- Time fields (`starttime`, `stoptime`) parsed as strings
- Medication identity fields parsed as strings
- Dose- and frequency-related fields exhibit float parsing due to missingness

Mixed dtypes reflect raw medication order complexity.
No schema corrections applied at this stage.

In [21]:
prescriptions_df.isna().sum()

subject_id              0
hadm_id                 0
pharmacy_id             0
poe_id                  1
poe_seq                 1
order_provider_id       0
starttime               0
stoptime                0
drug_type               0
drug                    0
formulary_drug_cd       0
gsn                    80
ndc                     0
prod_strength           0
form_rx              1000
dose_val_rx             0
dose_unit_rx            0
form_val_disp           0
form_unit_disp          0
doses_per_24_hrs      333
route                   0
dtype: int64

### Observation — prescriptions.csv missingness

- Core identifiers and linkage fields show no missing values
- Minimal missingness in order tracking fields (`poe_id`, `poe_seq`)
- High missingness in `form_rx`, reflecting optional or inconsistently recorded formulation data
- Partial missingness in `gsn` and `doses_per_24_hrs`, expected for heterogeneous medication orders

Missingness patterns reflect clinical and administrative variability, not data corruption.

### Red Flags & Decisions — prescriptions.csv

- Medication orders treated as treatment intent, not confirmed administration
- Mixed datatypes and missingness logged for intentional handling later
- Dose and formulation fields considered high-noise and used cautiously
- No transformations, filtering, or aggregation applied in this notebook

This table will be aggregated to admission-level treatment adaptation signals.

## File Inspection: diagnoses_icd.csv

### 1. Provenance & Structural Integrity

This inspection validates:
- Column structure and naming consistency
- Key linkage fields (`subject_id`, `hadm_id`)
- Multiplicity per admission
- Coding schema (ICD versioning)

In [22]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path(r"D:\mimic-iv-3.1\Unzipped_raw_data")

icustays_df = pd.read_csv(
    RAW_DIR / "icustays.csv",
    nrows=1000
)

icustays_df.shape


(1000, 8)

In [23]:
icustays_df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los'],
      dtype='str')

### Inference — icustays.csv Structure

- The table is ICU-episode–level, not admission-level.
- Presence of `stay_id` indicates distinct ICU stays per admission.
- Multiple ICU stays may exist for a single `hadm_id`.
- Temporal fields (`intime`, `outtime`) define episode boundaries.
- Aggregation will be required before admission-level modeling.

In [24]:
icustays_df[["hadm_id", "stay_id"]].head(10)


,hadm_id,stay_id
0,29079034,39553978
1,25860671,37081114
2,26913865,39765666
3,24597018,37067082
4,27703517,34592300
5,25563031,31205490
6,26133978,39698942
7,26184834,37510196
8,23581541,39060235
9,27793700,34672098


In [25]:
# Data types inspection
icustays_df.dtypes

subject_id          int64
hadm_id             int64
stay_id             int64
first_careunit        str
last_careunit         str
intime                str
outtime               str
los               float64
dtype: object

### Inference — icustays.csv Data Types

- Identifier fields (`subject_id`, `hadm_id`, `stay_id`) are numeric and structurally consistent.
- ICU location fields are categorical (`first_careunit`, `last_careunit`).
- Temporal fields (`intime`, `outtime`) are currently strings and require explicit datetime parsing later.
- `los` is numeric (`float64`), suggesting precomputed ICU length of stay.
- No implicit type corruption is visible at the raw inspection stage.

In [26]:
icustays_df.isna().sum()

subject_id        0
hadm_id           0
stay_id           0
first_careunit    0
last_careunit     0
intime            0
outtime           0
los               0
dtype: int64

### Inference — icustays.csv Missingness

- No missing values detected across sampled rows.
- All identifier fields are complete.
- Temporal boundaries (`intime`, `outtime`) are fully populated.
- ICU location fields are consistently recorded.
- The table appears structurally complete and internally consistent at the raw level.


In [27]:
icustays_df.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252
2,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
4,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113


### Inference — icustays.csv Raw Content Inspection

- Temporal fields use ISO datetime string format and require explicit parsing.
- `los` appears to represent ICU length of stay in days (decimal precision).
- ICU unit transitions are captured via `first_careunit` and `last_careunit`.
- Each row represents a bounded ICU episode with defined entry and exit times.
- Structure is consistent with ICU escalation modeling at the episode level.


In [28]:
(icustays_df["outtime"] > icustays_df["intime"]).head()


0    True
1    True
2    True
3    True
4    True
dtype: bool

### Inference — Temporal Ordering Consistency

- `outtime` is consistently greater than `intime` in sampled rows.
- ISO datetime formatting preserves chronological ordering even as strings.
- No immediate structural anomalies in ICU episode boundaries are observed.
- Temporal fields are internally coherent at the raw level.
- Safe to proceed with later explicit datetime parsing.


## File Inspection: transfers.csv

### 1. Provenance & Structural Integrity
We inspect care transition structure, linkage to admissions, and episode granularity.

In [29]:
transfers_df = pd.read_csv(
    RAW_DIR / "transfers.csv",
    nrows=1000
)

transfers_df.shape


(1000, 7)

### Observation — transfers.csv (Sample: 1,000 rows)

- Sampled 1,000 transfer records successfully.
- Total columns: 7.
- Likely represents intra-hospital movement events.
- Row-level meaning appears event-based rather than admission-level.
- Further inspection required to determine linkage structure.


In [30]:
transfers_df.columns


Index(['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit',
       'intime', 'outtime'],
      dtype='str')

### Inference — transfers.csv Structure

- The table is transfer-event–level, not admission-level.
- `transfer_id` indicates each row is a distinct care transition.
- `eventtype` categorizes movement type within hospital flow.
- Temporal fields define bounded care unit intervals.
- One admission (`hadm_id`) may contain multiple transfer events.


In [31]:
transfers_df.dtypes

subject_id       int64
hadm_id        float64
transfer_id      int64
eventtype          str
careunit           str
intime             str
outtime            str
dtype: object

In [32]:
transfers_df["hadm_id"].isna().sum()


np.int64(167)

### Inference — transfers.csv Identifier Integrity

- `hadm_id` is inferred as float due to presence of missing values.
- Approximately 16–17% of sampled transfer rows lack an associated admission ID.
- This suggests some transfer events may not be linked to a hospital admission.
- `transfer_id` remains structurally intact as the row-level key.
- Admission-level filtering will require explicit handling of missing `hadm_id`.

In [33]:
transfers_df.isna().sum()

subject_id       0
hadm_id        167
transfer_id      0
eventtype        0
careunit         0
intime           0
outtime        231
dtype: int64

### Inference — transfers.csv Missingness

- Missing `hadm_id` values indicate some transfer events are not linked to admissions.
- Missing `outtime` suggests some care unit episodes are open-ended or administratively incomplete.
- The table captures hospital flow events that are broader than admission-level structure.
- Filtering strictly on `hadm_id` will exclude unlinked transfer events.
- Admission-level modeling must account for partial linkage in this table.

In [34]:
transfers_df.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,UNKNOWN,2180-05-07 17:21:27,NaN
3,10000032,22841357.0,34100253,discharge,UNKNOWN,2180-06-27 18:49:12,NaN
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12


### Inference — transfers.csv Event Structure

- `eventtype` encodes hospital state transitions (e.g., ED, admit, discharge).
- Discharge events are associated with missing `outtime` values.
- Some discharge rows have `careunit = UNKNOWN`, suggesting terminal administrative states.
- Transfer events represent hospital flow transitions rather than stable care episodes.
- The table models movement and status changes within a hospitalization timeline.


In [35]:
transfers_df["eventtype"].value_counts()

eventtype
ED           336
admit        232
discharge    231
transfer     201
Name: count, dtype: int64

### Inference — transfers.csv Event Distribution

- Transfer events encode structured hospital flow states.
- ED entries are the most frequent event type in the sample.
- Admission and discharge events are nearly balanced.
- Intra-hospital transfers represent a significant portion of events.
- The table captures full admission trajectories, not isolated movements.


## File Inspection: procedures_icd.csv

### 1. Provenance & Structural Integrity

We inspect procedural coding structure and multiplicity per admission.

This inspection validates:
- Column structure and naming consistency
- Linkage via `subject_id` and `hadm_id`
- Procedure coding system and versioning
- Expected one-to-many relationship per admission

No filtering, transformation, or aggregation is performed at this stage.
The objective is to understand the raw structural contract of the file.


In [36]:
procedures_df = pd.read_csv(
    RAW_DIR / "procedures_icd.csv",
    nrows=1000
)

procedures_df.shape


(1000, 6)

### Observation — procedures_icd.csv (Sample: 1,000 rows)

- Sampled 1,000 procedure records successfully.
- Total columns: 6.
- Likely long-form structure with multiple procedures per admission.
- Further inspection required to validate coding structure.

In [37]:
procedures_df.columns

Index(['subject_id', 'hadm_id', 'seq_num', 'chartdate', 'icd_code',
       'icd_version'],
      dtype='str')

### Inference — procedures_icd.csv Structure

- The table is procedure-level, not admission-level.
- `seq_num` captures ordering of procedures within an admission.
- `icd_code` and `icd_version` indicate ICD-based procedural coding.
- `chartdate` provides temporal context for the procedure.
- Multiple procedure rows per `hadm_id` are structurally expected.


In [38]:
procedures_df.dtypes

subject_id     int64
hadm_id        int64
seq_num        int64
chartdate        str
icd_code         str
icd_version    int64
dtype: object

### Inference — procedures_icd.csv Data Types

- Identifier fields (`subject_id`, `hadm_id`) are structurally intact as integers.
- `seq_num` preserves procedural ordering within admission.
- `icd_code` is string-encoded, appropriate for alphanumeric codes.
- `icd_version` is numeric and differentiates ICD coding systems.
- `chartdate` is currently a string and requires explicit datetime parsing later.

In [39]:
procedures_df.isna().sum()

subject_id     0
hadm_id        0
seq_num        0
chartdate      0
icd_code       0
icd_version    0
dtype: int64

### Inference — procedures_icd.csv Missingness

- No missing values detected across sampled rows.
- All identifier and linkage fields are complete.
- Procedural coding fields (`icd_code`, `icd_version`) are fully populated.
- Temporal field (`chartdate`) is consistently recorded.
- The table is structurally complete at the raw inspection stage.


In [40]:
procedures_df.head()

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


### Inference — ICD Code Format Variation

- ICD-9 procedure codes are primarily numeric.
- ICD-10 procedure codes are structured alphanumeric strings.
- `icd_version` correctly distinguishes between coding systems.
- Mixed code formats are expected and structurally valid.
- `icd_code` must remain string-typed to preserve ICD-10 integrity.

## File Inspection: emar.csv

### 1. Provenance & Structural Integrity

We inspect medication administration event structure and linkage to admissions.

This inspection validates:
- Event-level granularity
- Linkage via `subject_id` and `hadm_id`
- Temporal structure of medication actions
- Completeness of administration metadata

No filtering or preprocessing is performed at this stage.

In [41]:
emar_df = pd.read_csv(
    RAW_DIR / "emar.csv",
    nrows=1000
)

emar_df.shape

(1000, 12)

### Observation — emar.csv (Sample: 1,000 rows)

- Sampled 1,000 medication administration records successfully.
- Total columns: 12.
- Likely represents event-level medication administration actions.
- Further inspection required to validate linkage and temporal structure.

In [42]:
emar_df.columns

Index(['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id',
       'enter_provider_id', 'charttime', 'medication', 'event_txt',
       'scheduletime', 'storetime'],
      dtype='str')

### Inference — emar.csv Structure

- The table is medication administration–event level.
- `emar_id` and `emar_seq` define granular administration records.
- Multiple identifiers link medication orders and pharmacy systems.
- Temporal fields (`charttime`, `scheduletime`, `storetime`) capture different workflow stages.
- Each admission (`hadm_id`) may contain numerous medication administration events.

In [43]:
emar_df.dtypes

subject_id             int64
hadm_id              float64
emar_id                  str
emar_seq               int64
poe_id                   str
pharmacy_id          float64
enter_provider_id        str
charttime                str
medication               str
event_txt                str
scheduletime             str
storetime                str
dtype: object

In [44]:
emar_df.isna().sum()

subject_id             0
hadm_id               72
emar_id                0
emar_seq               0
poe_id                 0
pharmacy_id          269
enter_provider_id    935
charttime              0
medication            21
event_txt              4
scheduletime           0
storetime              0
dtype: int64

### Inference — emar.csv Missingness & Identifier Structure

- `hadm_id` is partially missing, leading to float type inference.
- High missingness in `enter_provider_id` reflects workflow variability.
- `pharmacy_id` is not consistently populated across events.
- Core event timestamps are complete and structurally reliable.
- The table captures granular medication workflow events beyond strict admission-level scope.

In [45]:
emar_df["event_txt"].value_counts().head(10)

event_txt
Administered                   710
Flushed                         75
Not Given                       72
Not Given per Sliding Scale     42
Not Flushed                     21
Started                         15
Confirmed                       14
Stopped                         11
Delayed Administered             7
Hold Dose                        5
Name: count, dtype: int64

## Check for Mixed Types in Raw IDs

In [5]:
import pandas as pd
from pathlib import Path

RAW = Path(r"D:\mimic-iv-3.1\Unzipped_raw_data")

admissions = pd.read_csv(
    RAW / "admissions.csv",
    dtype={"subject_id": "Int64", "hadm_id": "Int64"}
)

diagnoses = pd.read_csv(
    RAW / "diagnoses_icd.csv",
    dtype={
        "subject_id": "Int64",
        "hadm_id": "Int64",
        "icd_code": "string",
        "icd_version": "Int64"
    }
)

print("Loaded.")


Loaded.


In [6]:
print(admissions["subject_id"].dtype)
print(admissions["hadm_id"].dtype)

print(diagnoses["subject_id"].dtype)
print(diagnoses["hadm_id"].dtype)


Int64
Int64
Int64
Int64


In [7]:
print("Admissions null subject_id:", admissions["subject_id"].isna().sum())
print("Admissions null hadm_id:", admissions["hadm_id"].isna().sum())

print("Diagnoses null subject_id:", diagnoses["subject_id"].isna().sum())
print("Diagnoses null hadm_id:", diagnoses["hadm_id"].isna().sum())


Admissions null subject_id: 0
Admissions null hadm_id: 0
Diagnoses null subject_id: 0
Diagnoses null hadm_id: 0


In [8]:
print("ICD code dtype:", diagnoses["icd_code"].dtype)
print("Null ICD codes:", diagnoses["icd_code"].isna().sum())

print("Unique ICD versions:", diagnoses["icd_version"].unique())


ICD code dtype: string
Null ICD codes: 0
Unique ICD versions: <IntegerArray>
[9, 10]
Length: 2, dtype: Int64


In [9]:
diagnoses["icd_code"].apply(type).value_counts()


icd_code
<class 'str'>    6364488
Name: count, dtype: int64